In [ ]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage #所有信息的父类
from langchain_core.messages import ToolMessage #在调用tool后将信息传回给LLM
from langchain_core.messages import SystemMessage #为LLM提供指导的信息（prompting）
from langchain_community.llms import Tongyi
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
load_dotenv()


True

In [ ]:
email =  Annotated[str, "Email address of the user."]
print(email.__metadata__)
# Squence 自动更新序列


('Email address of the user.',)


### Reducer Function（向后添加而不是覆盖）
- Rule that controls how updates from nodes are combinedwith the existingstate.
- Tells us how to merge new data into the current state
- Without a reducer, updates would have replaced the existing value entirely!

- Without a reducer
state={"messages": ["Hi!"]}
update =
new_state=
{"messageb":["Nice to meet you!"]}
messages":["Nicetomeetyou!"]}

- With a reducer
state
update
new state
'messages":["Hi!"]}
messages
["Nice to meet you!"]}
messages":["Hi!", "Nice to meet you!"]}

In [3]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]


In [4]:
@tool
def add(a: int, b: int):
    """Add two numbers."""
    return a + b

tools = [add]


In [5]:
model = ChatTongyi(model="qwen-plus").bind_tools(tools)

In [6]:
def model_call(state:AgentState) -> AgentState:
    system_prompt = SystemMessage(content="You are a helpful assistant.")

    response = model.invoke([system_prompt] + state["messages"])
    return {"messages":[response]}

In [7]:
def should_continue(state:AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"

In [8]:
graph = StateGraph(AgentState) 
graph.add_node("our_agent",model_call)

tool_node = ToolNode(tools=tools)
graph.add_node("tools",tool_node)

graph.set_entry_point("our_agent")

graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue": "tools",
        "end": END,
    }
)
graph.add_edge("tools", "our_agent")

app = graph.compile()

In [9]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [11]:
inputs = {"messages":[("user","add 3 + 4.")]}
print_stream(app.stream(inputs,stream_mode="value"))  # or "all" for all messages
